<a href="https://colab.research.google.com/github/BokyungChoi/2019_Summer/blob/master/Automated_Hypothesis_Testing_with_Large_Dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setups

## Base setups

In [0]:
import sys
import os
import re
import gc
import time
import math
import json
import base64
import datetime
import itertools

import pandas as pd
import numpy as np
from functools import reduce

import matplotlib.pyplot as plt
import seaborn as sns

# setups only for colabotory usage
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
# from google.oauth2 import service_account # for service account

In [0]:
# for bigquery connection
project_id = 'type in your bigquery project id'
client=bigquery.Client(project_id)

In [0]:
# for Korean matplotlib on colabotory
import matplotlib.font_manager as fm 

# Korean font installation
!apt-get update -qq
!apt-get install fonts-nanum* -qq
!python --version

def current_font():
  print(f"설정 폰트 글꼴: {plt.rcParams['font.family']}, 설정 폰트 사이즈: {plt.rcParams['font.size']}")  # 파이썬 3.6 이상 사용가능
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
fm._rebuild()

Python 3.6.8
NanumGothic Eco


## Query

In [0]:
%%time

query="""
"""

raw_df = client.query(query).to_dataframe()
print(raw_df.shape)

(2321031, 31)
CPU times: user 4min 10s, sys: 6.26 s, total: 4min 16s
Wall time: 12min 12s


## Get your own treatment / control group dataframe



In [0]:
print("Treatment group 전체:", tmt.shape)
print("Treatment group Inorganic:", tmt_io.shape)
print("Treatment group Organic:", tmt_o.shape)
print("Treatment group Non:", tmt_non.shape)
print("Control group 전체:", ctl.shape)
print("Control group Inorganic:", ctl_io.shape)
print("Control group Organic:", ctl_o.shape)
print("Control group Non:", ctl_non.shape)

Treatment group 전체: (1807, 32)
Treatment group Inorganic: (197, 32)
Treatment group Organic: (1601, 32)
Treatment group Non: (9, 32)
Control group 전체: (40291, 32)
Control group Inorganic: (7789, 32)
Control group Organic: (31979, 32)
Control group Non: (523, 32)


# Plotting & Testing

## 1. Raw data - Plotting (interactive)


#### Treatment

In [0]:
import seaborn as sns
from google.colab import widgets
from scipy.stats import norm

# write down your list of columns you want to compare between treatment group / control group
lst=[]

def dist_draw(df,x):
    sns.set(rc={'figure.figsize':(8,6)})
    sns.distplot(df[x], hist=True, kde=True, bins=int(180/5), color = 'darkblue', hist_kws={'edgecolor':'black'},kde_kws={'linewidth': 4})   
    print("Skewness: %f" % df[x].skew())
    print("Kurtosis: %f" % df[x].kurt())
    
columns_idx=tmt[lst].columns
columns_lst=list(columns_idx)

tb = widgets.TabBar([str(i) for i in columns_lst],location='start')
for i in range(len(columns_lst)):
  # Only select the first 3 tabs, and render others in the background.
  with tb.output_to(i, select=(i < 1)):
    dist_draw(tmt[lst],columns_lst[i])


### Control

In [0]:
import seaborn as sns
from google.colab import widgets
from scipy.stats import norm

# write down your list of columns you want to compare between treatment group / control group
lst=[]

def dist_draw(df,x):
    sns.set(rc={'figure.figsize':(8,6)})
    sns.distplot(df[x], hist=True, kde=True, bins=int(180/5), color = 'darkblue', hist_kws={'edgecolor':'black'},kde_kws={'linewidth': 4})   
    print("Skewness: %f" % df[x].skew())
    print("Kurtosis: %f" % df[x].kurt())
    
columns_idx=ctl[lst].columns
columns_lst=list(columns_idx)

tb = widgets.TabBar([str(i) for i in columns_lst],location='start')
for i in range(len(columns_lst)):
  # Only select the first 3 tabs, and render others in the background.
  with tb.output_to(i, select=(i < 1)):
    dist_draw(ctl[lst],columns_lst[i])


## 2. Skewed Data Transformation








### Description

#### Standard functions used for such conversions include Normalization, the Sigmoid, Log, Cube Root and the Hyperbolic Tangent. It all depends on what one is trying to accomplish. Transform skewed data and convert it into values between 0 and 1 using Normalization function.



### Automated data transformation

In [0]:
# transform your selected columns(which are in lst) with three types of methods
# and finalize your columns with transformed into the best type of transformation method (where the sum of absolute skewness of treatment group & control group is closest to 0)

# sigmoid function
def sigmoid(x):
    e = np.exp(1)
    y = 1/(1+e**(-x))
    return y

# cube root function
def cube_root(df):
    return df**(1/3)

# normalizing at final step
def normalize(i):
    upper = i.max()
    lower = i.min()
    norm_series = (i - lower)/(upper-lower)
    return norm_series

# integrated
def transform(df1,df2,lst):
  
     # random sampling 100 if you want it before transformation function
#     df1=df1.sample(n=100)
#     df2=df2.sample(n=100)
  
    for y in lst:
    # sigmoid check
        sigmoid_series1=sigmoid(df1[y])
        sigmoid_series2=sigmoid(df2[y])

        # log+1 check
        log1_series1=np.log1p(df1[y])
        log1_series2=np.log1p(df2[y])

        # cube root check
        cube_series1=cube_root(df1[y])
        cube_series2=cube_root(df2[y])

        # organize the functions
        options1=[sigmoid_series1,log1_series1,cube_series1]
        options2=[sigmoid_series2,log1_series2,cube_series2]
        options_name=['sigmoid function','log1p function','cube root function']

        # randomly set 
        best_skew_both=100 
        best_function_name='none'

        # comparing
        for ser1,ser2, func_name in zip(options1,options2,options_name):
        
        # normalizing in function if you both want normalization transformation & transformation with function
    #       ser1=normalize(ser1)
    #       ser2=normalize(ser2)

          if abs(best_skew_both) > abs(ser1.skew())+abs(ser2.skew()):
            best_skew_both=abs(ser1.skew())+abs(ser2.skew())
            best_function_name=func_name
            best_series1=ser1
            best_series2=ser2


        print(y,'is best when transformed into',best_function_name) 
        print('first df skewness: ',best_series1.skew())
        print('second df skewness: ',best_series2.skew())
        print("")
    
        # transforming; by only typing the function, it will change your original dataframes
        df1[y]=ser1
        df2[y]=ser2
       

### Usage

In [0]:
# write down your list of columns you want to compare between treatment group / control group
lst=[]

transform(tmt,ctl,lst)

reserve_count is best when transformed into cube root function
first df skewness:  1.0270930846491715
second df skewness:  0.2421802495927067

rev_total_avg is best when transformed into cube root function
first df skewness:  2.133780850320439
second df skewness:  0.10507008801015286



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in log1p
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log1p
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value 

contribution_margin_avg is best when transformed into cube root function
first df skewness:  2.310719073569172
second df skewness:  0.2376810130041142

visit_day_count is best when transformed into log1p function
first df skewness:  -0.6367206762897272
second df skewness:  -0.20768584012910885

PREVIEW_CAR_RENTAL_count_af is best when transformed into cube root function
first df skewness:  -0.19760063960976204
second df skewness:  -0.09773425967089361



## 3. Transformed data - Plotting (interactive)




#### Treatment

In [0]:
import seaborn as sns
from google.colab import widgets
from scipy.stats import norm

# write down your list of columns you want to compare between treatment group / control group
lst=[]

def dist_draw(df,x):
    sns.set(rc={'figure.figsize':(8,6)})
    sns.distplot(df[x], hist=True, kde=True, bins=int(180/5), color = 'darkblue', hist_kws={'edgecolor':'black'},kde_kws={'linewidth': 4})   
    print("Skewness: %f" % df[x].skew())
    print("Kurtosis: %f" % df[x].kurt())
    
columns_idx=tmt[lst].columns
columns_lst=list(columns_idx)

tb = widgets.TabBar([str(i) for i in columns_lst],location='start')
for i in range(len(columns_lst)):
  # Only select the first 3 tabs, and render others in the background.
  with tb.output_to(i, select=(i < 1)):
    dist_draw(tmt[lst],columns_lst[i])


### Control

In [0]:
import seaborn as sns
from google.colab import widgets
from scipy.stats import norm

# write down your list of columns you want to compare between treatment group / control group
lst=[]

def dist_draw(df,x):
    sns.set(rc={'figure.figsize':(8,6)})
    sns.distplot(df[x], hist=True, kde=True, bins=int(180/5), color = 'darkblue', hist_kws={'edgecolor':'black'},kde_kws={'linewidth': 4})   
    print("Skewness: %f" % df[x].skew())
    print("Kurtosis: %f" % df[x].kurt())
    
columns_idx=ctl[lst].columns
columns_lst=list(columns_idx)

tb = widgets.TabBar([str(i) for i in columns_lst],location='start')
for i in range(len(columns_lst)):
  # Only select the first 3 tabs, and render others in the background.
  with tb.output_to(i, select=(i < 1)):
    dist_draw(ctl[lst],columns_lst[i])


## 4. Normality test, Equal variance test, Independent T-test, Mann Whitney U test

### Description ( in Korean )

#### 정규성 검정 후 정규 분포 따를 시 Bartlett 등분산 검정, 따르지 않을 시 Levene 등분산 검정 & Mann Whitney U test , 등분산 검정 후 등분산일 시 등분산 독립표본 T-검정, 등분산 따르지 않을 시 이분산 독립표본-T-검정


###  * t-test 를 이용한 캠페인 집단별 비교를 하기 전에!
> 정규성 검정
> * 귀무가설 ($H_0$): 데이터의 분포가 정규분포를 따른다
> * 대립가설 ($H_1$): 데이터의 분포가 정규분포를 따르지 않는다 

>  검정의 종류
> * Shapiro-Wilk Test: `from scipy.stats import shapiro`
> * Anderson-Darling Test: `from scipy.stats import shapiro`

> 주의할 점
> * 데이터의 갯수가 5,000개 이상이면 p-value 의 특성상 매우 작게 나올 수 있음. 이 때는 적당한 숫자의 표본을 다시 추출하여 검정을 하는 것이 바람직함. 
> * 일부 검정 함수의 경우 경고 메시지를 출력하기도 하니 자세히 살펴보는 게 좋을 듯
> *   참고자료: [A Gentle Introduction to Normality Tests in Python](https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/)

### * 비모수적 검정
> 비모수 방법은 언제 하나?
1. Transformation으로 해결이 안될 때
2. 표본수가 너무 적어(6이하) 분포를
알 수 없는 경우
3. 집단들의 표본수가 서로 크게 다를 경우
4. 변수가 명칭 혹은 순서 척도일 때
5. 중앙값의 비교가 목적일 때
6. 최소한의 가정. 즉 등분산성, 정규분포 등의
가정을 만족 못할 때

> 모수 : 비모수
> Independent t-test : Mann-Whitney test
> Paired t-test : Wilcoxon signed rank test


### Automated hypothesis testing

In [0]:
import scipy as sp
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import anderson
from scipy.stats import mannwhitneyu


# 입력 인자 - 실험군 데이터프레임, 대조군 데이터프레임, 비교하고자 하는 y지표 리스트, 'KS' 또는 'AD' 의 두 정규성 검정 방식, 희망 sampling 수(integer)
# Input - df1= treatment dataframe, df2 = control dataframe, lst= list of columns you want to compare, norm_option = 'KS' or 'AD', sample= number of samples
def integrated_test(df1,df2,lst,norm_option,sample):
    
    # 귀무가설 기각한 y지표를 for문 안에서 리스트화할 준비
    success=[]    
    
    # 검정 시작
    print("[실험군과 대조군 모집단에 대한 정규성 검정 %s / 등분산 검정/ 독립표본 T-검정]"%norm_option)
    print("")
    print("실험군 n: ", len(df1))
    print("대조군 n: ", len(df2))
    print("(Randomly sampled n=%s)"%sample)
    print("")

    for y in lst:
        print('- Y값이 %s일때'%y)
        
        # random sampling
        df1_y=df1[y].sample(n=sample,random_state=23)  
        df2_y=df2[y].sample(n=sample,random_state=23)
        
        
        # 1) 정규성 검정 Normality test
        
        # 1-1) Kolmogorov Smirnov test
        if norm_option=='KS':
            Nmresult_tmt=sp.stats.kstest(df1_y,'norm')
            print('\t실험군: KS Statistics: %.3f \n\tp-value : %.3f' % (Nmresult_tmt))
            print("")
            Nmresult_ctl=sp.stats.kstest(df2_y,'norm')
            print('\t대조군: KS Statistics: %.3f \n\tp-value : %.3f' % (Nmresult_ctl))
            print("")

            # 1의 결과 두 y지표 모두 정규분포 일때
            if (Nmresult_tmt[1]> 0.05) & (Nmresult_ctl[1]> 0.05) :

                 # 2) 등분산 검정 Equal variance test using Bartlett 
                print("\t두 모집단 모두 정규분포를 따름으로 가정하여, Barlett 등분산 검정을 진행한다.")
                btresult = stats.bartlett(df1_y, df2_y)
                print('\tBartlett Result(F) : %.3f \n\tp-value : %.3f' % (lresult)) # F값이란 집단간 분산/집단내 분산

                if (btresult[1]<=0.05) & (btresult[0] <= 100):

                    # 3) 독립표본 t-test 등분산 가정, Independent t-test with equal variance
                    print("\t결과: 두 모집단의 분산은 동일함을 가정하여, 등분산 독립표본 검정을 진행한다.")
                    print("")
                    Indresult=sp.stats.ttest_ind(df1_y, df_y, equal_var=True)
                    print('\tT-Statistics: %.3f \n\tp-value : %.3f' % (Indresult))
                    if Indresult[1]<=0.05:
                        print("\t결과: 귀무가설을 기각한다.")
                        success.append(y)
                    else:
                        print("\t결과: 귀무가설을 기각하지 않는다.")
                    print("")

                else:

                    # 3) 독립표본 t-test 등분산 가정 Independent t-test with unequal variance
                    print("\t결과: 두 모집단의 분산은 동일하지 않음을 가정하여, 이분산 독립표본 검정을 진행한다.")
                    print("")
                    Indresult=sp.stats.ttest_ind(df1_y, df2_y, equal_var=False)
                    print('\tT-Statistics: %.3f \n\tp-value : %.3f' % (Indresult))
                    if Indresult[1]<=0.05:
                        print("\t결과: 귀무가설을 기각한다.")
                        success.append(y)
                    else:
                        print("\t결과: 귀무가설을 기각하지 않는다.")
                    print("")

            # 두 y지표 다 정규분포 아닐 때        
            else:

                # 2) 등분산 검정 Equal variance test using levene
                print("\t두 모집단 모두 정규분포를 따르지 않음으로 가정하여, Levene 등분산 검정을 진행한다.")
                lresult = stats.levene(df1_y, df2_y)
                print('\tLevene Result(F) : %.3f \n\tp-value : %.3f' % (lresult)) # F값이란 집단간 분산/집단내 분산

                if (lresult[1]<=0.05) & (lresult[0] <= 100):

                    # 3) Mann Whitney U test 등분산 가정(doesn't really matter)
                    print("\t결과: 두 모집단의 분산은 동일함을 가정하고, Mann Whitney U 검정을 진행한다.")
                    print("")
                    Indresult=mannwhitneyu(df1_y, df2_y)
                    print('\tMWU-Statistics: %.3f \n\tp-value : %.3f' % (Indresult))
                    if Indresult[1]<=0.05:
                        print("\t결과: 귀무가설을 기각한다.")
                        success.append(y)
                    else:
                        print("\t결과: 귀무가설을 기각하지 않는다.")
                    print("")

                else:

                    # 3) Mann Whitney U 이분산 가정 (doesn't really matter)
                    print("\t결과: 두 모집단의 분산은 동일하지 않음을 가정하고, Mann Whitney U 검정을 진행한다.")
                    print("")
                    Indresult=mannwhitneyu(df1_y, df2_y)
                    print('\tMWU-Statistics: %.3f \n\tp-value : %.3f' % (Indresult))
                    if Indresult[1]<=0.05:
                        print("\t결과: 귀무가설을 기각한다.")
                        success.append(y)
                    else:
                        print("\t결과: 귀무가설을 기각하지 않는다.")
                    print("")

            print("")
            
            
        # 1-2) Anderson Darling Test
        elif norm_option=='AD':
            Nmresult_tmt = anderson(df1_y)
            Nmresult_ctl = anderson(df2_y)
            
            p = 0
            count_reject_fail_tmt=0 # 어느 신뢰수준이든 귀무가설 기각 실패시 count+=1을 하고, count가 1이상이면 정규분포로 고려
            count_reject_fail_ctl=0

            for i in range(len(Nmresult_tmt.critical_values)): # 신뢰수준은 여기서 바꿀 수 있다.
                sl_tmt, cv_tmt = Nmresult_tmt.significance_level[i], Nmresult_tmt.critical_values[i]
                sl_ctl, cv_ctl = Nmresult_ctl.significance_level[i], Nmresult_ctl.critical_values[i]
                
                if (Nmresult_tmt.statistic < Nmresult_tmt.critical_values[i]):
                    count_reject_fail_tmt +=1
                    print('\t\t(실험군 - 신뢰수준 %s에서 기각 기록)'%sl_tmt)
                if (Nmresult_ctl.statistic < Nmresult_ctl.critical_values[i]):
                    count_reject_fail_ctl +=1
                    print('\t\t(대조군 - 신뢰수준 %s에서 기각 기록)'%sl_ctl)

            # 정규 분포일때(df1, df2 모두에서 한 y지표에 대해 어느 신뢰수준이든 귀무가설 기각에 실패한 기록이 있을 때)
            if (count_reject_fail_tmt >=1) & (count_reject_fail_ctl >=1) :

                 # 2) 등분산 검정 Equal variance test using bartlett
                print("\t두 모집단 모두 정규분포를 따름으로 가정하여, Barlett 등분산 검정을 진행한다.")
                btresult = stats.bartlett(df1_y, df2_y)
                print('\tBartlett Result(F) : %.3f \n\tp-value : %.3f' % (lresult)) # F값이란 집단간 분산/집단내 분산

                if (btresult[1]<=0.05) & (btresult[0] <= 100):

                    # 3) 독립표본 t-test 등분산 가정 Independent t-test with equal variance
                    print("\t결과: 두 모집단의 분산은 동일함을 가정하여, 등분산 독립표본 검정을 진행한다.")
                    print("")
                    Indresult=sp.stats.ttest_ind(df1_y, df2_y, equal_var=True)
                    print('\tT-Statistics: %.3f \n\tp-value : %.3f' % (Indresult))
                    if Indresult[1]<=0.05:
                        print("\t결과: 귀무가설을 기각한다.")
                        success.append(y)
                    else:
                        print("\t결과: 귀무가설을 기각하지 않는다.")
                    print("")

                else:

                    # 3) 독립표본 t-test 이분산 가정 Independent t-test with unequal variance
                    print("\t결과: 두 모집단의 분산은 동일하지 않음을 가정하여, 이분산 독립표본 검정을 진행한다.")
                    print("")
                    Indresult=sp.stats.ttest_ind(df1_y, df2_y, equal_var=False)
                    print('\tT-Statistics: %.3f \n\tp-value : %.3f' % (Indresult))
                    if Indresult[1]<=0.05:
                        print("\t결과: 귀무가설을 기각한다.")
                        success.append(y)
                    else:
                        print("\t결과: 귀무가설을 기각하지 않는다.")
                    print("")

            # 정규분포 아닐 때 (df1, df2 모두에서 한 y지표에 대해 어느 신뢰수준이든 귀무가설 기각한 기록만 있을때)       
            else:
                
                # 2) 등분산 검정 Equal variance test using levene
                print("\t두 모집단 모두 정규분포를 따르지 않음으로 가정하여, Levene 등분산 검정을 진행한다.")
                lresult = stats.levene(df1_y, df2_y)
                print('\tLevene Result(F) : %.3f \n\tp-value : %.3f' % (lresult)) # F값이란 집단간 분산/집단내 분산

                if (lresult[1]<=0.05) & (lresult[0] <= 100):

                    # 3) Mann Whitney U 등분산 가정
                    print("\t결과: 두 모집단의 분산은 동일함을 가정하고, Mann Whitney U 검정을 진행한다.")
                    print("")
                    Indresult=mannwhitneyu(df1_y, df2_y)
                    print('\tMWU-Statistics: %.3f \n\tp-value : %.3f' % (Indresult))
                    if Indresult[1]<=0.05:
                        print("\t결과: 귀무가설을 기각한다.")
                        success.append(y)
                    else:
                        print("\t결과: 귀무가설을 기각하지 않는다.")
                    print("")

                else:

                    # 3) Mann Whitney U 이분산 가정
                    print("\t결과: 두 모집단의 분산은 동일하지 않음을 가정하고, Mann Whitney U 검정을 진행한다.")
                    print("")
                    Indresult=mannwhitneyu(df1_y, df2_y)
                    print('\tMWU-Statistics: %.3f \n\tp-value : %.3f' % (Indresult))
                    if Indresult[1]<=0.05:
                        print("\t결과: 귀무가설을 기각한다.")
                        success.append(y)
                    else:
                        print("\t결과: 귀무가설을 기각하지 않는다.")
                    print("")

            print("")

    print("유의미한 차이를 보인 y지표 : ",success)

### Usage

In [0]:
# write down your list of columns you want to compare between treatment group / control group
lst=[]

integrated_test(tmt,ctl,lst,'KS',100)

[실험군과 대조군 모집단에 대한 정규성 검정 KS / 등분산 검정/ 독립표본 T-검정]

실험군 n:  1807
대조군 n:  40291
(Randomly sampled n=100)

- Y값이 reserve_count일때
	실험군: KS Statistics: 0.841 
	p-value : 0.000

	대조군: KS Statistics: 0.831 
	p-value : 0.000

	두 모집단 모두 정규분포를 따르지 않음으로 가정하여, Levene 등분산 검정을 진행한다.
	Levene Result(F) : 1.975 
	p-value : 0.162
	결과: 두 모집단의 분산은 동일하지 않음을 가정하고, Mann Whitney U 검정을 진행한다.

	MWU-Statistics: 4365.500 
	p-value : 0.032
	결과: 귀무가설을 기각한다.


- Y값이 contribution_margin_avg일때
	실험군: KS Statistics: nan 
	p-value : nan

	대조군: KS Statistics: nan 
	p-value : nan

	두 모집단 모두 정규분포를 따르지 않음으로 가정하여, Levene 등분산 검정을 진행한다.
	Levene Result(F) : nan 
	p-value : nan
	결과: 두 모집단의 분산은 동일하지 않음을 가정하고, Mann Whitney U 검정을 진행한다.

	MWU-Statistics: 4502.000 
	p-value : 0.112
	결과: 귀무가설을 기각하지 않는다.


- Y값이 rev_total_avg일때
	실험군: KS Statistics: 1.000 
	p-value : 0.000

	대조군: KS Statistics: 0.990 
	p-value : 0.000

	두 모집단 모두 정규분포를 따르지 않음으로 가정하여, Levene 등분산 검정을 진행한다.
	Levene Result(F) : 2.604 
	p-value : 0.108
	결과: 두 모집단의 분산은 동일하지 않음을 

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [0]:
integrated_test(tmt,ctl,lst,'AD',100)

[실험군과 대조군 모집단에 대한 정규성 검정 AD / 등분산 검정/ 독립표본 T-검정]

실험군 n:  1807
대조군 n:  40291
(Randomly sampled n=100)

- Y값이 reserve_count일때
	두 모집단 모두 정규분포를 따르지 않음으로 가정하여, Levene 등분산 검정을 진행한다.
	Levene Result(F) : 1.975 
	p-value : 0.162
	결과: 두 모집단의 분산은 동일하지 않음을 가정하고, Mann Whitney U 검정을 진행한다.

	MWU-Statistics: 4365.500 
	p-value : 0.032
	결과: 귀무가설을 기각한다.


- Y값이 contribution_margin_avg일때
	두 모집단 모두 정규분포를 따르지 않음으로 가정하여, Levene 등분산 검정을 진행한다.
	Levene Result(F) : nan 
	p-value : nan
	결과: 두 모집단의 분산은 동일하지 않음을 가정하고, Mann Whitney U 검정을 진행한다.

	MWU-Statistics: 4502.000 
	p-value : 0.112
	결과: 귀무가설을 기각하지 않는다.


- Y값이 rev_total_avg일때
		(실험군 - 신뢰수준 15.0에서 기각 기록)
		(대조군 - 신뢰수준 15.0에서 기각 기록)
		(실험군 - 신뢰수준 10.0에서 기각 기록)
		(대조군 - 신뢰수준 10.0에서 기각 기록)
		(실험군 - 신뢰수준 5.0에서 기각 기록)
		(대조군 - 신뢰수준 5.0에서 기각 기록)
		(실험군 - 신뢰수준 2.5에서 기각 기록)
		(대조군 - 신뢰수준 2.5에서 기각 기록)
		(실험군 - 신뢰수준 1.0에서 기각 기록)
		(대조군 - 신뢰수준 1.0에서 기각 기록)
	두 모집단 모두 정규분포를 따름으로 가정하여, Barlett 등분산 검정을 진행한다.
	Bartlett Result(F) : nan 
	p-value : nan
	결과: 두 모집단의 분산은 동일하지 않음을

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1849: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1937: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
